# Deploy Retrieval Augmented Generation Function in Watson Machine Learning

This notebook handles the deployment of a Python function and its associated assets to a deployment space in Watson Machine Learning. 

**Prerequisites:**

1. Succesfully completed the prerequisites for `1_file_ingestion_from_cos` and ran the notebook succesfully
2. Create a deployment space in Watson Machine Learning and added the Space GUID to your `Notebook_and_Deployment_Parameters` parameter set in the `wml_space_id` field

<details>
<summary><i>Click for directions on creating a deployment space </i></summary>

Watson Machine Learning provides deployment spaces where the user can save, configure and deploy their functions or models. We can save models, functions and data assets in this space.
If you do not have a space already created, you can follow the steps below to create one:

1. Navigate to Deployments

2. Click New Deployment Space

3. Choose a deployment space name, and associate your Cloud Object Storage and Watson Machine Learning service to the space

4. Once the space is created, navigate to the "Manage" tab and copy the Space GUID
    
</details>


**Before running any cells, connect the notebook to the rest of the project by inserting a project token.**

You can do this by clicking the three vertical dots next to the Code Snippet icon on the top right of the notebook UI and then clicking Insert project token to insert the token at the beginning of your notebook. Make sure to execute the new cell at the top of the notebook before running any other notebook cells.

## Configure Notebook

The cells below configure the notebook by importing the necessary packages, connecting to the Watson Machine Learning client, and adding the values configured in your project's Parameter Sets to the namespace. Note that the following values within your parameter sets will affect the output of running this notebook

| Parameter Set Variable Name | Description |
| --- | --- |
| `es_query_num_results` | The number of results returned by the Elasticsearch query. |
| `es_model_name` | The name of the model in Elasticsearch that will be used for querying. |
| `run_evaluator` | Whether to run the LlamaIndex faithfulness evaluator on generated answers as a guardrail against hallucination. |

In [ ]:
import json
import pprint as pp
import re
from typing import Dict, Any

import ibm_watson_machine_learning

wslib.download_file("utils.py")
import utils

In [ ]:
wslib_extension = utils.WSLibExtension(wslib)
parameters = wslib_extension.get_all_parameter_set_values()
SPACE_ID = parameters["wml_space_id"]
IBM_CLOUD_APIKEY = parameters["ibm_cloud_apikey"]
IBM_CLOUD_REGION = parameters["ibm_cloud_region"]

In [ ]:
wml_credentials = {
    "url": f"https://{IBM_CLOUD_REGION}.ml.cloud.ibm.com",
    "apikey": IBM_CLOUD_APIKEY,
}
wml_client = ibm_watson_machine_learning.APIClient(wml_credentials)
wml_client.set.default_space(SPACE_ID)

## Promote Assets to the Deployment Space.

To deploy the Python function, a few assets in the project also need to be promoted to the deployment space in addition to the function itself. These include:

| File Name | Description |
| --- | --- |
| `Mixtral-RAG-Template` | The LLM Prompt template is promoted to the deployment space as a JSON |
| `utils.py` | This Python file contains utility functions used across by the deployed function |
| `Watsonx-Discovery` | This is the Watsonx-Discovery connection which contains authorization credentials to the Databases for Elasticsearch instance |
| `Notebook_and_Deployment_Parameters` | The parameter set in this project is also deployed to the deployment space |

In [ ]:
WXD_CONNECTION_NAME = "WatsonxDiscovery"
UTILITY_MODULE_FILE_NAME = "utils.py"
parameter_set_json = wslib_extension.get_all_parameter_set_values()
llm_parameters_json = wslib_extension.get_asset_as_json(
    asset_name="Mixtral-RAG-Template", asset_type="wx_prompt"
)
JSONS_TO_EXPORT = {
    "llm_parameters.json": llm_parameters_json,
    "other_rag_parameters.json": parameter_set_json,
}

In [ ]:
existing_wml_connections = {
    asset["entity"]["name"]: asset["metadata"]["id"]
    for asset in wml_client.connections.get_details()["resources"]
}
if WXD_CONNECTION_NAME in existing_wml_connections:
    print(f"Deleting existing connection {WXD_CONNECTION_NAME}")
    wml_client.connections.delete(existing_wml_connections[WXD_CONNECTION_NAME])

wxd_connection = wslib.get_connection(WXD_CONNECTION_NAME)
connection_metadata = wxd_connection.pop(".")
connection_id = wml_client.connections.get_datasource_type_uid_by_name("elasticsearch")
wxd_connection_details = wml_client.connections.create(
    {
        wml_client.connections.ConfigurationMetaNames.NAME: connection_metadata["name"],
        wml_client.connections.ConfigurationMetaNames.DESCRIPTION: connection_metadata[
            "description"
        ],
        wml_client.connections.ConfigurationMetaNames.DATASOURCE_TYPE: connection_id,
        wml_client.connections.ConfigurationMetaNames.PROPERTIES: wxd_connection,
    }
)
data_connections = {
    wxd_connection_details["entity"]["name"]: wxd_connection_details["metadata"]["id"]
}

In [ ]:
export_details = [
    utils.export_json_to_wml_deployment_space(json_data, file_name, wml_client)
    for file_name, json_data in JSONS_TO_EXPORT.items()
]
wml_asset_ids = {
    asset["metadata"]["name"]: asset["metadata"]["asset_id"] for asset in export_details
}

In [ ]:
existing_wml_assets = {
    asset["metadata"]["name"]: asset["metadata"]["guid"]
    for asset in wml_client.data_assets.get_details()["resources"]
}

existing_asset_id_to_delete = existing_wml_assets.get(UTILITY_MODULE_FILE_NAME)
if existing_asset_id_to_delete:
    print(f"Deleting existing asset {UTILITY_MODULE_FILE_NAME}")
    wml_client.data_assets.delete(existing_wml_assets[UTILITY_MODULE_FILE_NAME])
asset_details = wml_client.data_assets.create(
    UTILITY_MODULE_FILE_NAME, file_path=UTILITY_MODULE_FILE_NAME
)
wml_asset_ids[UTILITY_MODULE_FILE_NAME] = asset_details["metadata"]["asset_id"]

deployed_function_params = {
    "space_id": SPACE_ID,
    "ibm_api_key": IBM_CLOUD_APIKEY,
    "ibm_cloud_region": IBM_CLOUD_REGION,
    "data_assets": wml_asset_ids,
    "data_connections": data_connections,
}

## Promote and Deploy the Python Function to Perform RAG

Given a natural language query from a user, the Python function performs the following: 

1. Connects to the index in the Databases for Elasticsearch service which was set up in `1_file_ingestion_from_cos` and retrieves relevant documents to the query using Elasticsearch's embedded ELSER model
2. Connects to Watsonx.ai and generates a response using the retrieved documents based on the `Mixtral-RAG-Template` prompt template
3. If `run_evaluator` is set to `True`, checks against potential hallucination using LlamaIndex's [FaithfulnessEvaluator](https://docs.llamaindex.ai/en/stable/examples/evaluation/faithfulness_eval.html)

The schema for the input and output to the deployed function is provided below for convenience:
| Level | Key | Description |
| --- | --- | --- |
| Input | `fields` | This is a list of field names for the input data. In this case, there is only one field, "Text". |
|  | `values` | This is a list of lists containing the values for each field. Only a single user query should be provided in this nested list. |
| Output | `predictions[0][llm_response]` | This is the generated response to the user query based on the provided documents. |
|  | `references` | This is a list of reference documents used to generate the response. Each reference contains `id`, `text`, `score`, and `metadata` fields. |
|  | `evaluations` | This is a dictionary of LLM evaluation results configured within the function. |

In [ ]:
def watsonx_discovery_rag_function(params=deployed_function_params):

    ###############################################################
    #################### ON DEPLOYMENT STARTUP ####################
    ###############################################################

    import json

    from ibm_watson_machine_learning import APIClient
    from elasticsearch import AsyncElasticsearch
    from llama_index.core import VectorStoreIndex, PromptTemplate, Settings
    from llama_index.vector_stores.elasticsearch import ElasticsearchStore

    # Connect to WML
    ibm_api_key = params["ibm_api_key"]
    ibm_cloud_region = params["ibm_cloud_region"]
    space_id = params["space_id"]
    wml_credentials = {
        "url": f"https://{ibm_cloud_region}.ml.cloud.ibm.com",
        "apikey": ibm_api_key,
    }
    wml_client = APIClient(wml_credentials)
    wml_client.set.default_space(params["space_id"])

    # Download the data_assets in the space
    for file_name, asset_id in params["data_assets"].items():
        wml_client.data_assets.download(asset_id, file_name)

    from utils import CustomWatsonX, create_sparse_vector_query_with_model

    def load_assets_and_connections_from_space(wml_client: APIClient, params=params):
        wxd_creds = wml_client.connections.get_details(
            params["data_connections"]["WatsonxDiscovery"]
        )["entity"]["properties"]

        json_assets = dict()
        for file_name in params["data_assets"]:
            if file_name.endswith(".json"):
                with open(file_name, "r") as f:
                    json_assets[file_name] = json.load(f)

        return {"wxd_creds": wxd_creds, "json_assets": json_assets}

    # Load downloaded assets into namespace
    space_assets_and_connections = load_assets_and_connections_from_space(wml_client)
    wxd_creds = space_assets_and_connections["wxd_creds"]
    json_assets = space_assets_and_connections["json_assets"]
    llm_parameters = json_assets["llm_parameters.json"]
    other_rag_parameters = json_assets["other_rag_parameters.json"]

    index_name = other_rag_parameters["es_index_name"]
    index_text_field = other_rag_parameters["es_index_text_field"]
    model_name = other_rag_parameters["es_model_name"]
    num_results = other_rag_parameters["es_query_num_results"]
    use_evaluator = other_rag_parameters["run_evaluator"] == "True"

    # Configure llama-index
    prompt_template = PromptTemplate(llm_parameters["input"][0][0])
    Settings.llm = CustomWatsonX(
        credentials=wml_credentials,
        space_id=space_id,
        model_id=llm_parameters["model_id"],
        validate_model_id=False,
        additional_kwargs=llm_parameters["model_parameters"],
    )
    Settings.embed_model = None

    # Configure optional evaluators
    evaluators = []
    if use_evaluator:
        from llama_index.core.evaluation import FaithfulnessEvaluator

        evaluators.append(FaithfulnessEvaluator())

    ###############################################################
    ################ ON DEPLOYMENT SCORING REQUEST ################
    ###############################################################

    def create_query_engine():
        async_es_client = AsyncElasticsearch(
            wxd_creds["url"],
            basic_auth=(wxd_creds["username"], wxd_creds["password"]),
            verify_certs=False,
        )
        vector_store = ElasticsearchStore(
            es_client=async_es_client,
            index_name=index_name,
            text_field=index_text_field,
        )
        index = VectorStoreIndex.from_vector_store(vector_store=vector_store)
        query_engine = index.as_query_engine(
            text_qa_template=prompt_template,
            similarity_top_k=num_results,
            vector_store_query_mode="sparse",
            vector_store_kwargs={
                "custom_query": create_sparse_vector_query_with_model(model_name)
            },
        )
        return query_engine

    def score(payload):
        try:
            user_query = payload["input_data"][0]["values"][0][0]
            elasticsearch_query_engine = create_query_engine()
            response = elasticsearch_query_engine.query(user_query)

            references = [
                {
                    **node.node.to_dict(
                        include={
                            "metadata",
                            "text",
                            "start_char_idx",
                            "end_char_idx",
                            "id_",
                        }
                    ),
                    "score": node.score,
                }
                for node in response.source_nodes
            ]
            eval_results = [
                evaluator.evaluate_response(response=response)
                for evaluator in evaluators
            ]
            scoring_response = {
                "predictions": [{"llm_response": response.response}],
                "references": references,
                "evaluations": {
                    evaluator.__class__.__name__: result.passing
                    for evaluator, result in zip(evaluators, eval_results)
                },
            }
            return scoring_response

        except Exception as e:
            return {"predictions": [{"error": str(e)}]}

    return score

#### Test the scoring function locally before deployment

In [ ]:
test_score = watsonx_discovery_rag_function()
test_question = "What are some key features of Watsonx.governance?"
test_payload = utils.format_scoring_input(test_question)

print(f"Scoring Request: {test_payload}")
print(f"Scoring Response:")
pp.pprint(test_score(test_payload))

#### Deployment of function to Space

In [ ]:
# Clean up existing deployment, function, and software specification
package_extension_file = "conda.yml"
package_extension_name = "llama-index package extension"
base_software_spec = "runtime-23.1-py3.10"
software_spec_name = f"{base_software_spec} with {package_extension_name}"
function_name = "watsonx_discovery_rag_function"
deployment_name = f"{function_name}_deployment"

existing_deployments = {
    deployment_details["metadata"]["name"]: wml_client.deployments.get_id(
        deployment_details
    )
    for deployment_details in wml_client.deployments.get_details()["resources"]
}
deployment_id_to_delete = existing_deployments.get(deployment_name)
if deployment_id_to_delete:
    wml_client.deployments.delete(deployment_id_to_delete)

existing_functions = {
    function_details["metadata"]["name"]: wml_client._functions.get_uid(
        function_details
    )
    for function_details in wml_client._functions.get_details()["resources"]
}
function_id_to_delete = existing_functions.get(function_name)
if function_id_to_delete:
    wml_client._functions.delete(function_id_to_delete)

software_spec_id_to_delete = wml_client.software_specifications.get_uid_by_name(
    software_spec_name
)
if software_spec_id_to_delete != "Not Found":
    wml_client.software_specifications.delete(software_spec_id_to_delete)

In [ ]:
# Upload Package Extension from Project
wslib.download_file(package_extension_file)
pe_asset_details = wml_client.package_extensions.store(
    meta_props={
        wml_client.package_extensions.ConfigurationMetaNames.NAME: package_extension_name,
        wml_client.package_extensions.ConfigurationMetaNames.TYPE: "conda_yml",
    },
    file_path=package_extension_file,
)
pe_asset_id = wml_client.package_extensions.get_id(pe_asset_details)

# Create Software Specification from Package Extension
base_id = wml_client.software_specifications.get_uid_by_name(base_software_spec)
ss_metadata = {
    wml_client.software_specifications.ConfigurationMetaNames.NAME: software_spec_name,
    wml_client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {
        "guid": base_id
    },
    wml_client.software_specifications.ConfigurationMetaNames.PACKAGE_EXTENSIONS: [
        {"guid": pe_asset_id}
    ],
}
ss_asset_details = wml_client.software_specifications.store(meta_props=ss_metadata)
ss_asset_id = wml_client.software_specifications.get_id(ss_asset_details)

# Store RAG function in WML
meta_data = {
    wml_client.repository.FunctionMetaNames.NAME: function_name,
    wml_client.repository.FunctionMetaNames.DESCRIPTION: "Python function to perform RAG using WatsonxDiscovery and Watsonx.ai",
    wml_client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: ss_asset_id,
}
watsonx_function_details = wml_client.repository.store_function(
    meta_props=meta_data, function=watsonx_discovery_rag_function
)
watsonx_function_uid = wml_client.repository.get_function_uid(watsonx_function_details)

In [ ]:
# Deploy the stored RAG function
meta_props = {
    wml_client.deployments.ConfigurationMetaNames.NAME: deployment_name,
    wml_client.deployments.ConfigurationMetaNames.HARDWARE_SPEC: {"name": "S"},
    wml_client.deployments.ConfigurationMetaNames.ONLINE: {},
}

wml_deployment_details = wml_client.deployments.create(
    watsonx_function_uid, meta_props=meta_props
)
wml_deployment_id = wml_client.deployments.get_uid(wml_deployment_details)

## Test the deployed endpoint by scoring some sample data

In [ ]:
wx_result = wml_client.deployments.score(
    wml_deployment_id, utils.format_scoring_input(test_question)
)
pp.pprint(wx_result)
utils.display_wml_response(test_question, wx_result)

## Update project assets

### Update the OpenAPI json with the new wml_deployment_id

In [ ]:
def replace_placeholders_in_JSON(
    json_dict: Dict[str, Any], replacements: Dict[str, str]
) -> Dict[str, Any]:
    """
    Replaces placeholders in a JSON object with corresponding values
    from a replacements dictionary.

    Args:
        json_dict (Dict[str, Any]): The JSON object containing placeholders.
        replacements (Dict[str, str]): A mapping of placeholder names to the
            values to replace them with

    Returns:
        Dict[str, Any]: The JSON object with placeholders
        replaced by their corresponding values.
    """
    json_string = json.dumps(json_dict)
    placeholders = re.findall(r"{{(.*?)}}", json_string)

    for placeholder in placeholders:
        if placeholder in replacements:
            json_string = json_string.replace(
                "{{" + placeholder + "}}", replacements[placeholder]
            )

    completed_json = json.loads(json_string)
    return completed_json


assistant_openapi_file_name = "watsonx-assistant-extension-openapi-template.json"
assistant_openapi_data = json.loads(
    wslib.load_data(assistant_openapi_file_name).read().decode("utf-8")
)
configured_openapi = replace_placeholders_in_JSON(
    assistant_openapi_data,
    replacements={
        "wml_deployment_id": wml_deployment_id,
        "ibm_cloud_region": IBM_CLOUD_REGION,
    },
)

wslib.save_data(
    "configured-watsonx-assistant-extension-openapi.json",
    json.dumps(configured_openapi, indent=4).encode(),
    overwrite=True,
)